In [38]:

import cv2
import numpy as np
from matplotlib import pyplot as plt

imgo = cv2.imread("NewImage-1.png", cv2.IMREAD_GRAYSCALE)
img = cv2.threshold(imgo,128,255,cv2.THRESH_BINARY)[1]
print(img)
print(img.shape)
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('img', 600, 400)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [255 255 255 255 255   0   0 255   0   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0   0   0 255   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]]
(10, 10)


In [39]:
def split_img(img,block_h,block_w):
    blocks = []
    h , w = img.shape[:2]
    if h%block_h!=0:
        print("invalid block height")
        return 
    if w%block_w!=0:
        print("invalid block width")
        return
    num_rows = w//block_w
    num_cols = h//block_h
    for y in range(num_cols):
        for x in range(num_rows):
                left = x * block_w
                upper = y * block_h
                right = left + block_w
                lower = upper + block_h
                block = img[upper:lower, left:right]
                blocks.append(block)
    return blocks

In [40]:
def CAC_encode(img,block_h,block_w):    
    blocks = split_img(img, block_w, block_h)
    #print(blocks)
    print("Image size:",img.size)
    codes = ['0','11','10']
    code = []
    for block in blocks:
        if np.all(block == 0):
            code.append(codes[0])
        elif np.all(block == 255):
            code.append(codes[1])
        else:
            code.append(codes[2])
            block = block.tolist()
            for blk in block:
                for b in blk:
                    if b == 255:
                        code.append(1)   
                    else:
                        code.append(b)
    code ="".join(map(str,code))
    print("Code length: ", len(code))
    
    CR = img.size/len(code)
    RD = 1 - 1/CR
    print("Compression Ratio: " , CR , " \nRelative Data Redundancy: ", RD) 
    return  block_h ,block_w , img.shape[0], img.shape[1],  code , CR , RD

In [41]:
block_h ,block_w , height,width, code = CAC_encode(img,5,5)[:5]

Image size: 100
Code length:  32
Compression Ratio:  3.125  
Relative Data Redundancy:  0.6799999999999999


In [42]:
def CAC_reblock(code,block_h,block_w,height,width):
    codes = ['0','11','10']
    blocks = []
    i = 0
    while i in range(len(code)) :
        if code[i] == codes[0]:
            blocks.append(np.zeros((block_h, block_w), dtype=np.uint8))
            i=i+1
        elif code[i:i+2] == codes[1]:
            blocks.append(np.ones((block_h, block_w), dtype=np.uint8) * 255)
            i = i+2
        elif code[i:i+2] == codes[2]: 
            i=i+2
            arr = np.zeros((block_h, block_w), dtype=np.uint8)
            for r in range(0,block_h):
                for c in range(0,block_w):
                    arr[r][c] = int(code[i])
                    i = i+1
            blocks.append(arr * 255)
    print(blocks)

In [43]:
def CAC_decode(code,block_h,block_w,height,width):
    img = np.ones((height, width), dtype=np.uint8)
    codes = ['0','11','10']
    blocks = []
    i = 0
    x = 0
    y = 0
    while i in range(len(code)):
        if x >= width:
            y = y + block_h 
            x = 0
        
        if code[i] == codes[0]:
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = 0
            x = x + block_w
            i = i+1
            
        elif code[i:i+2] == codes[1]:
            i = i+2
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = 255
            x= x + block_w
            
        elif code[i:i+2] == codes[2]: 
            i = i+2
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = int(code[i]) *255
                    i = i+1
            x = x + block_w
    return img

In [44]:
decoded_img = CAC_decode(code,block_h,block_w,height,width)
print(decoded_img)

decoded_img = cv2.cvtColor(decoded_img, cv2.COLOR_GRAY2BGR)
cv2.namedWindow('decoded_img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('decoded_img', 600, 400)
cv2.imshow('decoded_img', decoded_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [255 255 255 255 255   0   0 255   0   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0   0   0 255   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]]


In [45]:
#generate random valid block sizes 
def populate(h,w,n):
    pop=[]
    while len(pop) < n:
        block_h = np.random.randint(1,  h)
        block_w = np.random.randint(1,  w)
        if h% block_h == 0 and w% block_w == 0:
            pop.append([block_h,block_w])
    return pop

In [46]:

def fitness(img,population):
    fitness=[]
    for entry in population:
        print(entry)
        fitness.append(CAC_encode(img,entry[0],entry[1])[5])
    
    return sorted(zip(population, fitness), key=lambda x: x[1], reverse=True)


In [47]:
#crossover best n blocks
#mutate children
#def crossover(p1,p2):   
#loop

In [48]:
pop = populate(img.shape[0],img.shape[1],5)
print(pop)
print(fitness(img,pop))

[[2, 2], [1, 1], [5, 5], [5, 1], [1, 5]]
[2, 2]
Image size: 100
Code length:  86
Compression Ratio:  1.1627906976744187  
Relative Data Redundancy:  0.14
[1, 1]
Image size: 100
Code length:  167
Compression Ratio:  0.5988023952095808  
Relative Data Redundancy:  -0.6700000000000002
[5, 5]
Image size: 100
Code length:  32
Compression Ratio:  3.125  
Relative Data Redundancy:  0.6799999999999999
[5, 1]
Image size: 100
Code length:  45
Compression Ratio:  2.2222222222222223  
Relative Data Redundancy:  0.55
[1, 5]
Image size: 100
Code length:  60
Compression Ratio:  1.6666666666666667  
Relative Data Redundancy:  0.4
[([5, 5], 3.125), ([5, 1], 2.2222222222222223), ([1, 5], 1.6666666666666667), ([2, 2], 1.1627906976744187), ([1, 1], 0.5988023952095808)]
